In [ ]:
import pandas as pd
import numpy as np
import efficientnet.keras as efn
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model

In [ ]:
poster_dir = "../data/posters/"
poster_df = pd.read_csv("../data/MovieGenre.csv", encoding = "ISO-8859-1")
poster_df = poster_df.drop_duplicates(subset=['imdbId'], keep="last")
poster_df["Genre"] = poster_df["Genre"].str.split("|")
poster_df["filename"] = poster_df["imdbId"].astype(str) + ".jpg"
poster_df = poster_df[poster_df["Genre"].apply(lambda t: isinstance(t, list))]

In [ ]:
datagen = ImageDataGenerator(rescale=1./255., rotation_range=40, width_shift_range=0.2, 
                                   height_shift_range=0.2, horizontal_flip=True, validation_split=0.25)

train_generator = datagen.flow_from_dataframe(dataframe=poster_df,
                                            directory=poster_dir,
                                            x_col="filename",
                                            y_col="Genre",
                                            subset="training",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(182, 268))


valid_generator=datagen.flow_from_dataframe(dataframe=poster_df,
                                            directory=poster_dir,
                                            x_col="filename",
                                            y_col="Genre",
                                            subset="validation",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(182, 268))

In [ ]:
model = efn.EfficientNetB0(input_shape = (182, 268, 3), include_top = False, weights = 'imagenet')

for layer in model.layers:
    layer.trainable = False
    
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(28, activation="softmax")(x)
model_final = Model(inputs = model.input, outputs = predictions)

model_final.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model_final.fit(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=valid_generator,
        validation_steps=800)

Reference:
1. https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/
2. https://godatadriven.com/blog/keras-multi-label-classification-with-imagedatagenerator/